In [1]:
import numpy as np

import pandas as pd

from sklearn.model_selection import KFold

from sklearn.metrics import mean_squared_error

import xgboost as xgb

import math

/Library/Python/2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
INPUT_PATH = '../input/'
OUTPUT_PATH = '../output/'


def load_data():
    train = pd.read_csv(INPUT_PATH + 'trainv1.csv')
    train['target'] = train['target'].map(lambda x: data_log(x))
    submit = pd.read_csv(INPUT_PATH + 'submitv1.csv')
    return train, submit


def get_xgb_imp(xgb, feat_names):
    from numpy import array
    imp_vals = xgb.get_fscore()
    imp_dict = {feat_names[i]:float(imp_vals.get('f'+str(i),0.)) for i in range(len(feat_names))}
    total = array(imp_dict.values()).sum()
    return {k:v/total for k,v in imp_dict.items()}


def merge_dic(dicts):
    ret = {}
    for dict in dicts:
        for key in dict:
            val = dict[key]
            ret[key] = ret[key]+val if key in ret else val
    return ret

def data_log(x):
    if x <= 0:
        return 0
    else:
        return np.math.log(x + 1, 5)

In [3]:
print('begin to load data')
train, submit = load_data()

kfold = 10
skf = KFold(n_splits=kfold,shuffle=True, random_state=42)

begin to load data


In [4]:
params = {
        'objective': 'reg:linear',
        'max_depth': 6,
        'colsample_bytree': 0.8,
        'subsample': 0.8,
        'eta': 0.025,
        'gamma': 1,
        'reg_alpha': 0.5,
        'reg_lambda': 0.8,
        'eval_metric': 'rmse',
        'min_child_weight': 2,
        'silent': 1,
        'nthread': 6
    }



In [5]:
# cum = get_column_by_cum(train)
cum= ['comsume_counts_sum',
 'comsume_amounts_sum',
 'click_counts_sum']

# cum= []

all_features = [x for x in train.columns if not x in ['uid', 'target', 'active_date' ]+cum]

X = train.drop(['uid', 'target', 'active_date']+cum, axis=1).values
y = train.target.values

sub_id = submit.uid.values
submit = submit.drop(['uid', 'active_date']+cum, axis=1)

sub = pd.DataFrame()
sub['uid'] = sub_id
sub['target'] = np.zeros_like(sub_id)

scores = []

importances = []

In [37]:
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    # Convert our data into XGBoost format
    d_train = xgb.DMatrix(X_train, y_train)
    d_valid = xgb.DMatrix(X_valid, y_valid)
    d_submit = xgb.DMatrix(submit.values)
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    mdl = xgb.train(params, d_train, 4000, watchlist, early_stopping_rounds = 70, verbose_eval = 50)

    f_importance = get_xgb_imp(mdl,all_features)
    print(f_importance)
    importances.append(f_importance)

    score_this = np.sqrt(mean_squared_error(mdl.predict(d_valid, ntree_limit = mdl.best_ntree_limit), y_valid))
    print(score_this)
    scores.append(score_this)

    print('[Fold %d/%d Prediciton:]' % (i + 1, kfold))
    # Predict on our test data
    p_test = mdl.predict(d_submit, ntree_limit=mdl.best_ntree_limit)

    sub['target'] += p_test / kfold

[0]	train-rmse:2.45882	valid-rmse:2.47471
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 70 rounds.
[50]	train-rmse:1.91881	valid-rmse:1.91747
[100]	train-rmse:1.85429	valid-rmse:1.85253
[150]	train-rmse:1.84221	valid-rmse:1.84421
[200]	train-rmse:1.83605	valid-rmse:1.84232
[250]	train-rmse:1.83177	valid-rmse:1.84156
[300]	train-rmse:1.82812	valid-rmse:1.84133
[350]	train-rmse:1.82484	valid-rmse:1.84118
[400]	train-rmse:1.82159	valid-rmse:1.84122
[450]	train-rmse:1.81857	valid-rmse:1.8411
[500]	train-rmse:1.81551	valid-rmse:1.84102
[550]	train-rmse:1.81273	valid-rmse:1.84098
Stopping. Best iteration:
[508]	train-rmse:1.81506	valid-rmse:1.84093

{'consume_amount_cum': 0.087933703082677672, 'active_month': 0.037211402037920803, 'actived_months': 0.03827550625564298, 'click_count': 0.083032374564684638, 'comsume_count': 0.065619760092867283, 'age': 0.026634851025409519, 'click_count_cum': 0.072939507287

1.84204528819
[Fold 6/10 Prediciton:]
[0]	train-rmse:2.46181	valid-rmse:2.44763
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 70 rounds.
[50]	train-rmse:1.91654	valid-rmse:1.92955
[100]	train-rmse:1.85122	valid-rmse:1.87543
[150]	train-rmse:1.83905	valid-rmse:1.86992
[200]	train-rmse:1.83318	valid-rmse:1.86877
[250]	train-rmse:1.82885	valid-rmse:1.86799
[300]	train-rmse:1.82535	valid-rmse:1.86769
[350]	train-rmse:1.82201	valid-rmse:1.86744
[400]	train-rmse:1.81881	valid-rmse:1.86742
Stopping. Best iteration:
[373]	train-rmse:1.82051	valid-rmse:1.86718

{'consume_amount_cum': 0.08169528074977192, 'active_month': 0.04138674628846313, 'actived_months': 0.039562080119432692, 'click_count': 0.077589781869453436, 'comsume_count': 0.061292195405158831, 'age': 0.025545326366426143, 'click_count_cum': 0.075682176329103429, 'plannum': 0.052334743302645766, 'sex': 0.0048934229078543582, 'loan_count': 0.0464045

In [38]:
imp_sum = merge_dic(importances)
sort_rec = sorted(imp_sum.items(), key=lambda x:x[1])
print(sort_rec)

print('cv avg scores %s' % np.mean(scores))

[('active_year', 0.026840661303611985), ('sex', 0.052262832792648914), ('age', 0.26939972548077545), ('limit', 0.33641924495531011), ('loan_count_cum', 0.37878328139153666), ('active_month', 0.38305006635609484), ('actived_months', 0.38964778110455323), ('loan_count', 0.43828781561282348), ('plannum', 0.51320082130702416), ('plannum_cum', 0.51922169252726547), ('comsume_count_cum', 0.61101346059702322), ('comsume_count', 0.63285670878820166), ('click_count_cum', 0.73763392963077923), ('click_count', 0.82476070495765852), ('consume_amount_cum', 0.86203483710053674), ('consume_amount', 0.92089968854912418), ('loan_amount', 1.0387975254728001), ('loan_amount_cum', 1.0648892220722321)]
cv avg scores 1.85079058977


In [53]:
scores

[1.9582315980044378,
 1.9469649895881047,
 1.961923709968435,
 1.9509096657266631,
 1.9717851262963768,
 1.9482733523629558,
 1.9507409453458249,
 1.9283989172430984,
 1.9324870916566361,
 1.9217251948081049]

In [39]:
from datetime import datetime
sub.to_csv(OUTPUT_PATH +"submission{}.csv".format(datetime.now().strftime('%Y%m%d_%H%M%S')), index=False)

In [326]:
all_features = [x for x in train.columns if not x in ['uid', 'target', 'active_date']]

X = train.drop(['uid', 'target', 'active_date'], axis=1)
y = train.target

In [348]:
xgb_params = {
        'objective': ['reg:linear'],
        'learning_rate': [0.025],
        'max_depth': [6],
        'min_child_weight': [2],
        'silent': [1],
        'subsample': [ 0.8],
        'colsample_bytree': [0.8],
        'n_estimators': [400],
        'gamma': [1,3],
        'reg_alpha': [0.5,0.8],
        'reg_lambda': [0.8,1.3] 
    }

In [349]:

from sklearn.model_selection import GridSearchCV

import xgboost as xgb


In [350]:
modelXgboost = xgb.XGBRegressor()

    # Run the grid search
grid_obj_xgb = GridSearchCV(modelXgboost, xgb_params, cv=KFold(n_splits=5, shuffle=True, random_state=42), n_jobs=2, verbose=2, refit=True,scoring='mean_squared_error')
grid_obj_xgb = grid_obj_xgb.fit(X, y)


Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] reg_alpha=0.5, colsample_bytree=0.8, silent=1, learning_rate=0.025, min_child_weight=2, n_estimators=400, subsample=0.8, reg_lambda=0.8, objective=reg:linear, max_depth=6, gamma=1 
[CV] reg_alpha=0.5, colsample_bytree=0.8, silent=1, learning_rate=0.025, min_child_weight=2, n_estimators=400, subsample=0.8, reg_lambda=0.8, objective=reg:linear, max_depth=6, gamma=1 


/Library/Python/2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Library/Python/2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Library/Python/2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Library/Python/2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


[CV]  reg_alpha=0.5, colsample_bytree=0.8, silent=1, learning_rate=0.025, min_child_weight=2, n_estimators=400, subsample=0.8, reg_lambda=0.8, objective=reg:linear, max_depth=6, gamma=1, total= 1.4min
[CV] reg_alpha=0.5, colsample_bytree=0.8, silent=1, learning_rate=0.025, min_child_weight=2, n_estimators=400, subsample=0.8, reg_lambda=0.8, objective=reg:linear, max_depth=6, gamma=1 
[CV]  reg_alpha=0.5, colsample_bytree=0.8, silent=1, learning_rate=0.025, min_child_weight=2, n_estimators=400, subsample=0.8, reg_lambda=0.8, objective=reg:linear, max_depth=6, gamma=1, total= 1.4min
[CV] reg_alpha=0.5, colsample_bytree=0.8, silent=1, learning_rate=0.025, min_child_weight=2, n_estimators=400, subsample=0.8, reg_lambda=0.8, objective=reg:linear, max_depth=6, gamma=1 


/Library/Python/2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Library/Python/2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Library/Python/2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Library/Python/2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


[CV]  reg_alpha=0.5, colsample_bytree=0.8, silent=1, learning_rate=0.025, min_child_weight=2, n_estimators=400, subsample=0.8, reg_lambda=0.8, objective=reg:linear, max_depth=6, gamma=1, total= 1.5min
[CV] reg_alpha=0.5, colsample_bytree=0.8, silent=1, learning_rate=0.025, min_child_weight=2, n_estimators=400, subsample=0.8, reg_lambda=0.8, objective=reg:linear, max_depth=6, gamma=1 
[CV]  reg_alpha=0.5, colsample_bytree=0.8, silent=1, learning_rate=0.025, min_child_weight=2, n_estimators=400, subsample=0.8, reg_lambda=0.8, objective=reg:linear, max_depth=6, gamma=1, total= 1.5min
[CV] reg_alpha=0.5, colsample_bytree=0.8, silent=1, learning_rate=0.025, min_child_weight=2, n_estimators=400, subsample=0.8, reg_lambda=1.3, objective=reg:linear, max_depth=6, gamma=1 


...

[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed: 28.4min


[CV]  reg_alpha=0.8, colsample_bytree=0.8, silent=1, learning_rate=0.025, min_child_weight=2, n_estimators=400, subsample=0.8, reg_lambda=1.3, objective=reg:linear, max_depth=6, gamma=3, total= 1.4min
[CV] reg_alpha=0.8, colsample_bytree=0.8, silent=1, learning_rate=0.025, min_child_weight=2, n_estimators=400, subsample=0.8, reg_lambda=1.3, objective=reg:linear, max_depth=6, gamma=3 


/Library/Python/2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Library/Python/2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Library/Python/2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Library/Python/2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


[CV]  reg_alpha=0.8, colsample_bytree=0.8, silent=1, learning_rate=0.025, min_child_weight=2, n_estimators=400, subsample=0.8, reg_lambda=1.3, objective=reg:linear, max_depth=6, gamma=3, total= 1.2min
[CV]  reg_alpha=0.8, colsample_bytree=0.8, silent=1, learning_rate=0.025, min_child_weight=2, n_estimators=400, subsample=0.8, reg_lambda=1.3, objective=reg:linear, max_depth=6, gamma=3, total= 1.2min


[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed: 29.7min finished


In [351]:
clf = grid_obj_xgb.best_estimator_
grid_obj_xgb.best_params_

{'colsample_bytree': 0.8,
 'gamma': 1,
 'learning_rate': 0.025,
 'max_depth': 6,
 'min_child_weight': 2,
 'n_estimators': 400,
 'objective': 'reg:linear',
 'reg_alpha': 0.5,
 'reg_lambda': 0.8,
 'silent': 1,
 'subsample': 0.8}

In [59]:

users = pd.read_csv(INPUT_PATH + 't_user.csv')
orders = pd.read_csv(INPUT_PATH + 't_order.csv')
loans = pd.read_csv(INPUT_PATH + 't_loan.csv')
loans_sum = pd.read_csv(INPUT_PATH + 't_loan_sum.csv')
clicks = pd.read_csv(INPUT_PATH + 't_click.csv')

In [64]:
loans.loan_amount = 5**loans.loan_amount
loans_sum.loan_sum = 5 ** loans_sum.loan_sum - 1
orders.price = 5 ** orders.price
orders.discount = 5 ** orders.discount - 1
users.limit = 5 ** users.limit -1


In [65]:
orders["consume_amount"] = orders["price"] * orders["qty"] - orders["discount"]

In [68]:
import matplotlib.pyplot as plt

In [69]:
plt.figure(figsize=(8,6))
plt.scatter(range(orders.shape[0]), np.sort(orders.consume_amount.values))
plt.xlabel('index', fontsize=12)
plt.ylabel('consume_amount', fontsize=12)
plt.show()

/System/Library/Frameworks/Python.framework/Versions/2.7/Extras/lib/python/matplotlib/collections.py:548: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == 'face':


In [91]:
ulimit = np.percentile(orders.consume_amount.values, 99.9)
llimit = 0


In [81]:
orders.consume_amount.isnull().values.any()

False

In [94]:
orders = orders[ orders.consume_amount > llimit ]
orders = orders[ orders.consume_amount < ulimit ]

In [80]:
orders.consume_amount = orders.consume_amount.fillna(0)

In [79]:
orders[orders.cate_id==33].price.describe()

count    395972.000000
mean        999.471488
std        1802.303636
min           1.000000
25%          30.000000
50%          89.000000
75%        1100.000000
max       69888.999996
Name: price, dtype: float64

In [92]:
ulimit

44036.000002161658

In [93]:
llimit

0

In [96]:
orders.shape

(5384276, 7)

In [97]:
5400778-5384276

16502

In [113]:
user_info = pd.read_csv(OUTPUT_PATH + 'user_info.csv')

In [119]:
for c in user_info.columns:
        user_info[c] = user_info[c].fillna(0)

In [122]:
for month in [8, 9, 10, 11, 12]:
        user_info = cummulate_data_by_month(user_info, month)

In [121]:
CC = '{}_comsume_count'
CM = '{}_consume_amount'
LM = '{}_loan_amount'
LC = '{}_loan_count'
CKC = '{}_click_count'
CUM = '_cum'
basic = ['uid', 'age', 'sex', 'active_date', 'limit']
TARGET = 'target'

In [123]:
list(user_info)

['uid',
 'age',
 'sex',
 'active_date',
 'limit',
 '8_comsume_count',
 '9_comsume_count',
 '10_comsume_count',
 '11_comsume_count',
 '8_consume_amount',
 '9_consume_amount',
 '10_consume_amount',
 '11_consume_amount',
 '8_loan_amount',
 '9_loan_amount',
 '10_loan_amount',
 '11_loan_amount',
 '8_loan_count',
 '9_loan_count',
 '10_loan_count',
 '11_loan_count',
 '8_click_count',
 '9_click_count',
 '10_click_count',
 '11_click_count',
 '8_comsume_count_cum',
 '8_consume_amount_cum',
 '8_loan_amount_cum',
 '8_loan_count_cum',
 '8_click_count_cum',
 '9_comsume_count_cum',
 '9_consume_amount_cum',
 '9_loan_amount_cum',
 '9_loan_count_cum',
 '9_click_count_cum',
 '10_comsume_count_cum',
 '10_consume_amount_cum',
 '10_loan_amount_cum',
 '10_loan_count_cum',
 '10_click_count_cum',
 '11_comsume_count_cum',
 '11_consume_amount_cum',
 '11_loan_amount_cum',
 '11_loan_count_cum',
 '11_click_count_cum',
 '12_comsume_count_cum',
 '12_consume_amount_cum',
 '12_loan_amount_cum',
 '12_loan_count_cum',
 '

In [44]:
all_features

['age',
 'sex',
 'limit',
 'comsume_count',
 'consume_amount',
 'loan_amount',
 'loan_count',
 'plannum',
 'click_count',
 'comsume_count_cum',
 'consume_amount_cum',
 'loan_amount_cum',
 'loan_count_cum',
 'click_count_cum',
 'plannum_cum',
 'active_month',
 'active_year',
 'actived_months']

In [43]:
train.head()

,uid,age,sex,active_date,limit,active_month,active_year,comsume_count,consume_amount,loan_amount,...,consume_amount_cum,loan_amount_cum,loan_count_cum,click_count_cum,plannum_cum,actived_months,comsume_counts_sum,comsume_amounts_sum,click_counts_sum,target
0,26308,30,1,2016-02-16,15000.000001,2,1,1.0,700.000000,2500.0,...,0.0,0.0,0.0,0.0,0.0,6,3.0,1283.000000,213.0,5.070593
1,78209,40,1,2016-02-21,5000.000000,2,1,38.0,8429.100000,0.0,...,0.0,0.0,0.0,0.0,0.0,6,86.0,12510.600000,0.0,0.000000
2,51930,35,1,2016-04-19,24999.999999,4,1,2.0,790.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,4,3.0,830.900000,35.0,6.405333
3,10113,25,1,2016-03-12,24999.999999,3,1,23.0,2251.800000,0.0,...,0.0,0.0,0.0,0.0,0.0,5,34.0,2775.000000,154.0,0.000000
4,17067,35,1,2016-02-16,15000.000001,2,1,51.0,19244.299999,0.0,...,0.0,0.0,0.0,0.0,0.0,6,148.0,33788.489999,62.0,0.000000


In [45]:
list(train)

['uid',
 'age',
 'sex',
 'active_date',
 'limit',
 'active_month',
 'active_year',
 'comsume_count',
 'consume_amount',
 'loan_amount',
 'loan_count',
 'plannum',
 'click_count',
 'comsume_count_cum',
 'consume_amount_cum',
 'loan_amount_cum',
 'loan_count_cum',
 'click_count_cum',
 'plannum_cum',
 'actived_months',
 'comsume_counts_sum',
 'comsume_amounts_sum',
 'click_counts_sum',
 'target']

In [50]:
train[train['uid']==26308]

,uid,age,sex,active_date,limit,active_month,active_year,comsume_count,consume_amount,loan_amount,...,consume_amount_cum,loan_amount_cum,loan_count_cum,click_count_cum,plannum_cum,actived_months,comsume_counts_sum,comsume_amounts_sum,click_counts_sum,target
0,26308,30,1,2016-02-16,15000.000001,2,1,1.0,700.0,2500.000000,...,0.0,0.0,0.0,0.0,0.0,6,3.0,1283.0,213.0,5.070593
90993,26308,30,1,2016-02-16,15000.000001,2,1,0.0,0.0,3500.000000,...,700.0,2502.0,2.0,55.0,2.0,7,3.0,1283.0,213.0,6.033893
181986,26308,30,1,2016-02-16,15000.000001,2,1,2.0,583.0,16500.000001,...,700.0,6004.0,4.0,87.0,4.0,8,3.0,1283.0,213.0,5.501180
